In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw
import torchvision.transforms as transforms

# Define input and output folders
input_folder = 'input/'  # Replace with your training images folder path
output_folder = 'output/'  # Replace with your output folder path

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Custom transform for Gaussian noise
class AddGaussianNoise:
    def __init__(self, mean=0.0, sigma=20.0):
        self.mean = mean
        self.sigma = sigma
    
    def __call__(self, img):
        img = img.convert('RGB')
        img_np = np.array(img).astype(np.float32)
        # Generate Gaussian noise
        noise = np.random.normal(self.mean, self.sigma, img_np.shape)
        img_np = img_np + noise
        # Clip values to valid range [0, 255]
        img_np = np.clip(img_np, 0, 255).astype(np.uint8)
        return Image.fromarray(img_np)

# Custom transform to add artifact lines (mimicking scanning errors)
class AddArtifactLines:
    def __init__(self, max_lines=3, line_width=20, base_color=(0, 0, 0), blur_sigma=2.5):
        self.max_lines = max_lines  # Max number of lines (1–3)
        self.line_width = line_width  # Thicker lines
        self.base_color = base_color  # Black for artifacts
        self.blur = transforms.GaussianBlur(kernel_size=7, sigma=blur_sigma)  # Blur for lines
    
    def __call__(self, img):
        # Convert to RGBA for transparency
        img = img.convert('RGBA')
        width, height = img.size
        
        # Create transparent overlay for drawing lines
        overlay = Image.new('RGBA', (width, height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(overlay)
        
        # Randomly decide number of lines (1–3)
        num_lines = np.random.randint(1, self.max_lines + 1)
        # Randomly decide if lines include both horizontal and vertical for crossed table
        has_crossed = np.random.choice([True, False], p=[0.5, 0.5])
        
        if has_crossed and num_lines >= 2:
            # Ensure at least one horizontal and one vertical line for crossed table
            orientations = ['horizontal', 'vertical']
            np.random.shuffle(orientations)
            orientations = orientations[:num_lines]
            if len(orientations) < num_lines:
                orientations += [np.random.choice(['horizontal', 'vertical']) for _ in range(num_lines - len(orientations))]
        else:
            # Random orientations
            orientations = [np.random.choice(['horizontal', 'vertical']) for _ in range(num_lines)]
        
        for orientation in orientations:
            # Randomly choose line style: solid or dotted
            is_dotted = np.random.choice([True, False], p=[0.5, 0.5])
            # Random base opacity for the line (50%–100%)
            base_alpha = np.random.randint(128, 255)
            
            if orientation == 'horizontal':
                y = np.random.randint(0, height)
                if is_dotted:
                    # Draw dotted line with varying opacity within segments
                    segment_length = 20
                    gap_length = 10
                    x = 0
                    while x < width:
                        x_end = min(x + segment_length, width)
                        # Gradient opacity within segment
                        for x_seg in range(x, x_end):
                            alpha = int(base_alpha * (1 - (x_seg - x) / segment_length))
                            alpha = max(128, min(alpha, 255))
                            draw.line((x_seg, y, x_seg + 1, y), fill=self.base_color + (alpha,), width=self.line_width)
                        x += segment_length + gap_length
                else:
                    # Draw solid line with gradient opacity
                    for x in range(width):
                        alpha = int(base_alpha * (1 - x / width))
                        alpha = max(128, min(alpha, 255))
                        draw.line((x, y, x + 1, y), fill=self.base_color + (alpha,), width=self.line_width)
            else:
                x = np.random.randint(0, width)
                if is_dotted:
                    # Draw dotted line with varying opacity
                    segment_length = 20
                    gap_length = 10
                    y = 0
                    while y < height:
                        y_end = min(y + segment_length, height)
                        for y_seg in range(y, y_end):
                            alpha = int(base_alpha * (1 - (y_seg - y) / segment_length))
                            alpha = max(128, min(alpha, 255))
                            draw.line((x, y_seg, x, y_seg + 1), fill=self.base_color + (alpha,), width=self.line_width)
                        y += segment_length + gap_length
                else:
                    # Draw solid line with gradient opacity
                    for y in range(height):
                        alpha = int(base_alpha * (1 - y / height))
                        alpha = max(128, min(alpha, 255))
                        draw.line((x, y, x, y + 1), fill=self.base_color + (alpha,), width=self.line_width)
        
        # Composite overlay onto image
        img = Image.alpha_composite(img, overlay)
        # Convert back to RGB and apply blur
        img = img.convert('RGB')
        img = self.blur(img)
        return img

# Define augmentation pipeline
augmentations = [
    transforms.RandomHorizontalFlip(p=1.0),  # Always flip horizontally
    transforms.RandomVerticalFlip(p=1.0),    # Always flip vertically
    transforms.GaussianBlur(kernel_size=5, sigma=(0.5, 2.0)),  # Blur to mimic test set
    AddArtifactLines(max_lines=3, line_width=20, base_color=(245, 245, 245), blur_sigma=3),  # Near-white artifact lines
    AddArtifactLines(max_lines=3, line_width=20, base_color=(0, 0, 0), blur_sigma=3), # black artifact lines
    AddGaussianNoise(mean=0.75, sigma=20.0)
]

# Optional: Upscale to 1000x1000 to match test set resolution
resize_transform = transforms.Resize((1000, 1000))  # Comment out if not needed

# Get list of image files in the input folder (supporting jpg, png, jpeg)
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# Counter for output file indexing
output_idx = 1

# Process each image
prefix = '00000'
for filename in image_files:
    extension = filename.split('.')[-1]
    input_path = os.path.join(input_folder, filename)
    image = Image.open(input_path).convert('RGB')  # Open and convert to RGB
    
    # Optionally upscale the image to 1000x1000
    image = resize_transform(image)  # Comment out if not needed
    
    # Save the original (or upscaled) image
    output_path = os.path.join(output_folder, f'augmented_{output_idx}.{extension}')
    image.save(output_path)
    print(f"Saved original/upscaled: {output_path}")
    output_idx += 1
    
    # Apply each augmentation and save
    for aug_idx, transform in enumerate(augmentations, start=1):
        # Apply the transform
        augmented_image = transform(image)
        
        # Save the augmented image with increasing index
        output_path = os.path.join(output_folder, f'augmented_{output_idx}.{extension}')
        augmented_image.save(output_path)
        
        print(f"Augmented and saved: {output_path} (Transform: {transform.__class__.__name__})")
        output_idx += 1

Saved original/upscaled: output/augmented_1.bmp
Augmented and saved: output/augmented_2.jpg (Transform: RandomHorizontalFlip)
Augmented and saved: output/augmented_3.jpg (Transform: RandomVerticalFlip)
Augmented and saved: output/augmented_4.jpg (Transform: GaussianBlur)
Augmented and saved: output/augmented_5.jpg (Transform: AddArtifactLines)
Augmented and saved: output/augmented_6.jpg (Transform: AddArtifactLines)
Augmented and saved: output/augmented_7.jpg (Transform: AddGaussianNoise)
Saved original/upscaled: output/augmented_8.bmp
Augmented and saved: output/augmented_9.jpg (Transform: RandomHorizontalFlip)
Augmented and saved: output/augmented_10.jpg (Transform: RandomVerticalFlip)
Augmented and saved: output/augmented_11.jpg (Transform: GaussianBlur)
Augmented and saved: output/augmented_12.jpg (Transform: AddArtifactLines)
Augmented and saved: output/augmented_13.jpg (Transform: AddArtifactLines)
Augmented and saved: output/augmented_14.jpg (Transform: AddGaussianNoise)
